In [57]:
import sqlite3

# 构造表头
def generate_table_head(conn, task_id):
    sql = "select sm.id, sm.name, sm.experiment_remark, \
    sm.creation_date from experiments sm where sm.task_id={} and sm.del_flag=0".format(task_id)
    sub_model_result = conn.execute(sql).fetchall()

    # 构造表头
    table_head = {}
    table_length = {}
    sub_model_param = {}

    for sub_model in sub_model_result:

        sql = "select mp.param_type, mp.name, mp.param_value from model_params mp where mp.experiment_id=%d" % (
            sub_model[0])
        model_param_result = conn.execute(sql)#.fetchall()
        for model_param in model_param_result:
            # 添加 table head
            if model_param[0] not in table_head:
                table_head[model_param[0]] = [model_param[1]]
            else:
                if model_param[1] not in table_head[model_param[0]]:
                    table_head[model_param[0]].append(model_param[1])

            # sub_model_param
            if sub_model[0] not in sub_model_param:
                sub_model_param[sub_model[0]] = {model_param[1]: model_param[2]}
            else:
                sub_model_param[sub_model[0]][model_param[1]] = model_param[2]

    for type, param_list in table_head.items():
        table_length[type] = len(param_list)

    return sub_model_result, sub_model_param, table_head, table_length


# 构造页面数据
def generate_table_data(conn, table_head, sub_model_result, sub_model_param):
    table_data = []
    best_head = []
    best_data = {}
    first_param = {}
    modify_head = []
    id = 0
    for i, sub_model in enumerate(sub_model_result):

        # 评估指标数据
        sql = "select br.name, br.best_value from best_result br where br.experiment_id=%d" % (sub_model[0])
        best_result = conn.execute(sql)
        dic = {}

        flag = False
        for best in best_result:
            if best[0] not in best_head:
                best_head.append(best[0])

            dic[best[0]] = best[1]
            flag = True

        best_data[sub_model[0]] = dic

        id += 1
        dic = {}

        dic['id'] = id
        dic['sub_model_id'] = sub_model[0]
        dic['sub_model_name'] = sub_model[1]
        dic['sub_model_remark'] = sub_model[2]
        dic['create_time'] = sub_model[3]

        if flag:
            dic['finished_train'] = True
        else:
            dic['finished_train'] = False

        # 超参数
        for _, param_list in table_head.items():
            for param_name in param_list:

                try:
                    dic[param_name] = sub_model_param[sub_model[0]][param_name]
                except:
                    dic[param_name] = ''

                if i == 0:  # 记录第一次训练的超参数
                    first_param[param_name] = dic[param_name]
                else:
                    if dic[param_name] != first_param[param_name]:
                        modify_head.append(param_name)

        # 评估指标数据
        for name in best_head:
            try:
                dic[name] = best_data[sub_model[0]][name]
            except:
                dic[name] = ''

        table_data.append(dic)


    return table_data, best_head, modify_head


# 构造loss画图数据
def generate_loss_data(conn, task_id, sub_model_result):
    sql = "select max(md.epoch) from model_metrics md " \
          "left join experiments sm on sm.id=md.experiment_id " \
          "left join tasks m on m.id = sm.task_id " \
          "where m.id=%d" % (task_id)
    sql2 = "select md.epoch from model_metrics md " \
          "left join experiments sm on sm.id=md.experiment_id " \
          "left join tasks m on m.id = sm.task_id " \
          "where m.id=%d" % (task_id)
    max_step = conn.execute(sql).fetchall()[0][0]
    print(conn.execute(sql2).fetchall(),"ddddd")
    x_value = [i for i in range(1, max_step + 1)]

    legend = {}
    series = []
    for i, sub_model in enumerate(sub_model_result):

        if i + 1 == len(sub_model_result):
            legend[sub_model[1] + '_train'] = 'true'
            legend[sub_model[1] + '_test'] = 'true'
        else:
            legend[sub_model[1] + '_train'] = 'false'
            legend[sub_model[1] + '_test'] = 'false'

        sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='train_loss'" % (
        sub_model[0])
        train_value = [value[0] for value in conn.execute(sql)]
        data_dic = {'name': sub_model[1] + '_train', 'data': str(train_value)}
        series.append(data_dic)

        sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='test_loss'" % (
        sub_model[0])
        test_value = [value[0] for value in conn.execute(sql)]
        data_dic = {'name': sub_model[1] + '_test', 'data': str(test_value)}
        series.append(data_dic)

    x_value = str(x_value)

    return legend, x_value, series


def generate_new_loss(conn, experiment_id, experiment_name):

    sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='train_loss'" % (experiment_id)
    train_value = [value[0] for value in conn.execute(sql)]

    sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='test_loss'" % (
        experiment_id)
    test_value = [value[0] for value in conn.execute(sql)]

    data_list_loss = {'xAxis':{'data':[i for i in range(1, len(train_value) + 1)]},
                      'series':[{'name':experiment_name + '_train', 'data':train_value},
                                {'name':experiment_name + '_test', 'data':test_value}]}

    return data_list_loss


# 构造 acc画图数据 sub_model_result:experiment
def generate_indicater_data(conn, sub_model_result, type):
    legend_acc = {}
    series_acc = []
    for i, sub_model in enumerate(sub_model_result):

        if i + 1 == len(sub_model_result):
            legend_acc[sub_model[1] + '_test'] = 'true'
        else:
            legend_acc[sub_model[1] + '_test'] = 'false'

        sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='%s'" % (
        sub_model[0], type)
        test_value = [value[0] for value in conn.execute(sql)]
        data_dic = {'name': sub_model[1] + '_test', 'data': str(test_value)}
        series_acc.append(data_dic)

    return legend_acc, series_acc

def generate_new_indicater_data(conn, experiment_id, experiment_name, type):

    sql = "select md.metric_value from model_metrics md where md.experiment_id=%d and md.name='%s'" % (experiment_id, type)
    value_list = [value[0] for value in conn.execute(sql)]

    data_dict = {'xAxis': {'data': [i for i in range(1, len(value_list) + 1)]},
                      'series': [{'name': experiment_name + '_test', 'data': value_list}]}

    return data_dict

def get_experiment_list(conn):
    sql = "select m.task_id,b.name,b.creation_date from experiments m, tasks b where m.task_id=b.id"
    exp_result = conn.execute(sql).fetchall()
    return exp_result

# 删除model
def delete_model(conn, task_id):
    sql = "delete from tasks where id=%d" % (task_id)
    conn.execute(sql)
    conn.commit()


def delete_sub_model(del_set):
    sql = "delete from experiments where experiment_id in %s" % (del_set)
    conn = get_conn()
    conn.execute(sql)

    sql = "delete from model_params where experiment_id in %s" % (del_set)
    conn.execute(sql)

    sql = "delete from model_metrics where experiment_id in %s" % (del_set)
    conn.execute(sql)

    sql = "delete from best_result where experiment_id in %s" % (del_set)
    conn.execute(sql)

    conn.commit()
    conn.close()

## 模型总数
def get_model_num(conn):
    sql = "select count(1) from experiments sm"
    model_num = conn.execute(sql).fetchall()[0][0]
    return model_num

# 数据库连接
def get_conn(traking_path):
    return sqlite3.connect(traking_path)



cnn=get_conn("sql.db")
task_id = 2
sub_model_result, sub_model_param, table_head, table_length = generate_table_head(cnn,task_id)

table_data, best_head, modify_head = generate_table_data(cnn, table_head, sub_model_result, sub_model_param)

# 构造loss画图数据
#legend_loss, x_value_loss, series_loss = generate_loss_data(con, project_id_model, sub_model_result)

# 构造loss画图数据
legend, x_value, series = generate_loss_data(cnn, task_id, sub_model_result)

# loss
experiment_id =1
experiment_name = "test_track5"
data_list_loss = generate_new_loss(cnn, experiment_id, experiment_name)

# acc
sub_model_id = experiment_id
sub_model_name = experiment_name
data_list_acc = generate_new_indicater_data(cnn, sub_model_id, sub_model_name, 'test_acc')

# recall
data_list_recall = generate_new_indicater_data(cnn, sub_model_id, sub_model_name, 'test_recall')

# precision
data_list_precision = generate_new_indicater_data(cnn, sub_model_id, sub_model_name, 'test_precision')

# F1
data_list_F1 = generate_new_indicater_data(cnn, sub_model_id, sub_model_name, 'test_F1')



# 构造 acc画图数据
legend_acc, series_acc = generate_indicater_data(cnn, sub_model_result, 'test_acc')

# 构造recall画图数据
legend_recall, series_recall = generate_indicater_data(cnn, sub_model_result, 'test_recall')

# 构造 precision画图数据
legend_precision, series_precision = generate_indicater_data(cnn, sub_model_result, 'test_precision')

# 构造 F1画图数据
legend_F1, series_F1 = generate_indicater_data(cnn, sub_model_result, 'test_F1')


[(1,)] ddddd


In [54]:
sub_model_result, sub_model_param, table_head, table_length = generate_table_head(cnn,2)

table_data, best_head, modify_head = generate_table_data(cnn, table_head, sub_model_result, sub_model_param)


In [55]:
table_data, best_head, modify_head

([{'id': 1,
   'sub_model_id': 2,
   'sub_model_name': 'test_track6',
   'sub_model_remark': 'experiment_remark',
   'create_time': '2021-10-16 08:28:50.595045',
   'finished_train': True,
   'model': 'BILSTM_Attention',
   'optimizer': 'adam',
   'num_classes': '10',
   'lr': '0.001',
   'training_steps': '1000000',
   'display_step': '1',
   'batch_size': '100',
   'num_hidden': '128',
   'embedding_dim': '128',
   'drop_out': '0.5',
   'best_loss': 2.34},
  {'id': 2,
   'sub_model_id': 3,
   'sub_model_name': 'test_track7',
   'sub_model_remark': 'experiment_remark',
   'create_time': '2021-10-16 16:13:00.386830',
   'finished_train': False,
   'model': '',
   'optimizer': '',
   'num_classes': '',
   'lr': '',
   'training_steps': '',
   'display_step': '',
   'batch_size': '',
   'num_hidden': '',
   'embedding_dim': '',
   'drop_out': '',
   'best_loss': ''}],
 ['best_loss'],
 ['model',
  'optimizer',
  'num_classes',
  'lr',
  'training_steps',
  'display_step',
  'batch_size',


In [52]:
generate_table_head(cnn,2)

([(2, 'test_track6', 'experiment_remark', '2021-10-16 08:28:50.595045'),
  (3, 'test_track7', 'experiment_remark', '2021-10-16 16:13:00.386830')],
 {2: {'model': 'BILSTM_Attention',
   'optimizer': 'adam',
   'num_classes': '10',
   'lr': '0.001',
   'training_steps': '1000000',
   'display_step': '1',
   'batch_size': '100',
   'num_hidden': '128',
   'embedding_dim': '128',
   'drop_out': '0.5'}},
 {'tf_param': ['model',
   'optimizer',
   'num_classes',
   'lr',
   'training_steps',
   'display_step',
   'batch_size',
   'num_hidden',
   'embedding_dim',
   'drop_out']},
 {'tf_param': 10})

In [47]:
get_model_num(cnn)

2

In [45]:
legend_recall, series_recall

({'test_track5_test': 'true'}, [{'name': 'test_track5_test', 'data': '[]'}])

In [35]:
data_list_loss

{'xAxis': {'data': [1, 2, 3, 4]},
 'series': [{'name': 'test_track5_train', 'data': [24.0, 24.0, 24.0, 23.0]},
  {'name': 'test_track5_test', 'data': []}]}

In [33]:
legend, x_value, series

({'test_track5_train': 'true', 'test_track5_test': 'true'},
 '[1, 2]',
 [{'name': 'test_track5_train', 'data': '[24.0, 24.0, 24.0, 23.0]'},
  {'name': 'test_track5_test', 'data': '[]'}])

In [5]:
table_data, best_head, modify_head

([{'id': 1,
   'sub_model_id': 1,
   'sub_model_name': 'test_track5',
   'sub_model_remark': 'experiment_remark',
   'create_time': '2021-10-14 02:14:24.058297',
   'finished_train': True,
   'model': 'BILSTM_Attention',
   'optimizer': 'adam',
   'num_classes': 10.0,
   'lr': 0.001,
   'training_steps': 1000000.0,
   'display_step': 1.0,
   'batch_size': 100.0,
   'num_hidden': 128.0,
   'embedding_dim': 128.0,
   'drop_out': 0.5,
   'best_loss': 2.34}],
 ['best_loss'],
 [])

In [8]:
# 构造表头
project_id_model = get_project_list(conn)

[{'id': 1,
  'sub_model_id': 1,
  'sub_model_name': 'test_track5',
  'sub_model_remark': 'experiment_remark',
  'create_time': '2021-10-14 02:14:24.058297',
  'finished_train': True,
  'model': 'BILSTM_Attention',
  'optimizer': 'adam',
  'num_classes': 10.0,
  'lr': 0.001,
  'training_steps': 1000000.0,
  'display_step': 1.0,
  'batch_size': 100.0,
  'num_hidden': 128.0,
  'embedding_dim': 128.0,
  'drop_out': 0.5,
  'best_loss': 2.34}]

In [10]:
def get_experiment_list(conn):
    sql = "select m.name, m.creation_date, m.id from tasks m where m.del_flag = 0 order by m.creation_date desc"
    result = conn.execute(sql)
    project_list = []

    for item in result:
        map_ = {'project_name':item[0], 'create_time':item[1], 'project_id':item[2]}
        project_list.append(map_)
    if len(project_list)>0:
        return project_list[0]["project_id"]
    else:
        return 1
    
get_experiment_list(cnn)

OperationalError: no such column: m.del_flag

In [22]:
def get_experiment_list(conn):
    sql = "select m.name, m.task_id from experiments m"
    exp_result = conn.execute(sql).fetchall()
    exp_task_list = []
    for experiment_name,task_id in exp_result:
        sql2 = "select * from tasks m where m.id= {}".format(task_id)
        task_result = conn.execute(sql2).fetchone()
        exp_task_list.append(task_result)
    return exp_task_list
    
    
t = get_experiment_list(cnn)

In [23]:
for  i in t:
    print(i)

(5, '机器学习任务', 'leepand', '2021-10-15 09:28:05.955313', 2, '{}', '{}', '{}')
(2, 'my_task_tracking', 'leepand', '2021-10-14 02:14:24.039844', 1, '{}', '{}', '{}')


In [49]:
def get_experiment_list(conn):
    sql = "select m.task_id,b.name,b.creation_date from experiments m, tasks b where m.task_id=b.id"
    exp_result = conn.execute(sql).fetchall()
    return exp_result

t = get_experiment_list(cnn)
t[0][0]

5